In [1]:
import re
from copy import deepcopy
import json

import pandas as pd
import geopandas as gpd
from deepdiff import DeepDiff

import STARTHER
import nvdbapiv3
import nvdbgeotricks
import mangelrapport
import andremangler 
import objektdifferanser


Hurra, vi klarer importere nvdbapiv3


In [2]:
ls

LICENSE*                      doubleversjonanalyse.xlsx*
README.md*                    endreBKsekkepost.py*
STARTHER.py*                  gamlemangler/
__pycache__/                  mangelrapport-Bruksklasser2023-04-25.xlsx*
aarsrapport.py*               mangelrapport.py*
andremangler.py*              nvdbapi-clientinfo.json*
doubleVersions.ipynb*         pics/
doubleVersionsDetaljer.xlsx*  skrivkommunalBK.py*
doubleVersions_20230417.LOG*  skrivsmaahull.py*
doubleVersions_20230424.LOG*


In [10]:
# mydf = pd.DataFrame( mangelrapport.lesLoggfil( 'doubleVersions_20220922.LOG' )  )
mydf = pd.DataFrame( mangelrapport.lesLoggfil( 'doubleVersions_20230525.LOG' )  )


In [11]:
mydf['FeatureTypeId'].value_counts()

87    32
Name: FeatureTypeId, dtype: int64

### Fjerner spor (524) - og jevnhetsmåling (525)

In [12]:
mydf = mydf[ ~mydf['FeatureTypeId'].isin( [524, 525])]

## Fjerner et par som er fiksa

In [23]:
mydf = mydf[ ~mydf['FeatureId'].isin( [ 421037291, 915906260])]

In [24]:
len ( mydf['FeatureId'].unique() ) 

14

In [17]:
temp

,FeatureTypeId,FeatureId,VersionId,NetElemId,NStartPos,NEndPos,sdate,edate,vref,roadSysRef,municipality,sqldump_miljo,sqldump_dato,sqldump_klokke,sqldump_beskrivelse,sqldump_datarad,sqldump_datauttak,sqldump_parametre,filnavn
0,87,421037291,1,1672722,0.51909895,None,2013.06.26,None,7 _ EV18 HP89 14176 (dir=1),EV18 S33D1 m408 KD1 m176,3805,PRODUKSJON,2023-05-25,14:38,Lister ut alle objekter med mer enn en gyldig ...,0,Kjørt: 25.05.2023 - 03:32 - 03:35,Parametere:,doubleVersions_20230525.LOG
1,87,421037291,2,1672722,0.51909895,None,2013.08.08,None,7 _ EV18 HP89 14176 (dir=1),EV18 S33D1 m408 KD1 m176,3805,PRODUKSJON,2023-05-25,14:38,Lister ut alle objekter med mer enn en gyldig ...,1,Kjørt: 25.05.2023 - 03:32 - 03:35,Parametere:,doubleVersions_20230525.LOG


In [22]:
minFeil

'410 Gone : Feature 915906260, version 2 of type 87 gone!'

In [25]:
utenDuplikat = mydf.drop_duplicates( subset=['FeatureId', 'VersionId'] ).copy()
utenDuplikat.sort_values( by=[ 'FeatureId', 'VersionId'], ascending=True, inplace=True )

data = []
url = 'https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/' 
forb = nvdbapiv3.apiforbindelse()
forb.login( miljo='prodles')

params = { 'inkluder' : 'alle' }

mineID = mydf['FeatureId'].unique()
for minId in mineID: 
    print( f"Henter {minId} ")
    feilmeldinger = set()
    temp = utenDuplikat[ utenDuplikat['FeatureId'] == minId ]
    if len( temp ) != 2: 
        print( f"Fant {len(temp)} rader med FeatureID {minId} ")
    objekttype = temp.iloc[0]['FeatureTypeId']
    eldste_versjon = temp.iloc[0]['VersionId']
    eldste_start = andremangler.konverterdato( temp.iloc[0]['sdate'] )
    eldste_slutt = andremangler.konverterdato( temp.iloc[0]['edate'] ) 
    r = forb.les( url + str( objekttype ) + '/' + str(minId) + '/' + str(eldste_versjon), params=params  )
    if r.ok: 
        eldste = r.json()
    else: 
        feilmeldinger.add( objektdifferanser.dekodLesFeilmelding( r ) ) 
        

    nyeste_versjon = temp.iloc[1]['VersionId']
    nyeste_start = andremangler.konverterdato( temp.iloc[1]['sdate'] )
    nyeste_slutt = andremangler.konverterdato( temp.iloc[1]['edate'] ) 
    
    r = forb.les( url + str( objekttype ) + '/' + str(minId) + '/' + str(nyeste_versjon), params=params  )
    if r.ok: 
        nyeste = r.json()
    else: 
        feilmeldinger.add( objektdifferanser.dekodLesFeilmelding( r ) ) 

    # Henter objektet som kommer etter eldste i rekken
    eldste_pluss1_versjon = eldste_versjon + 1 
    r = forb.les( url + str( objekttype ) + '/' + str(minId) + '/' + str(eldste_pluss1_versjon), params=params  )
    if r.ok: 
        eldste_pluss1 = r.json()
        eldste_ny_sluttdato = eldste_pluss1['metadata']['startdato']
    else:     
        minFeil = objektdifferanser.dekodLesFeilmelding( r )
        if 'not found' in minFeil.lower(): 
            tidsvurdering = f"Ingen versjon {eldste_pluss1_versjon} av objektet"
        else: 
            feilmeldinger.add( minFeil )
        print( minFeil )

    kanLukkesEnkelt = False 
    # Sjekker at ny sluttdato ikke blir samme dag som startdato, eller at startdato > sluttdato
    if eldste_ny_sluttdato == eldste_start: 
        tidsvurdering = f"Kan ikke ha samme start- og sluttdato {eldste_start} : {minId} versjon {eldste_versjon} vs versjon {eldste_pluss1_versjon}  " 
    elif sorted( [ eldste_start, eldste_ny_sluttdato  ] )[-1] != eldste_ny_sluttdato: 
        tidsvurdering =  f"Objekt {minId} Startdato {eldste_ny_sluttdato} for v{eldste_pluss1_versjon}  er før startdato {eldste_start} v{eldste_versjon} " 
    else: 
        tidsvurdering =  f"HURRA, {minId} versjon {eldste_versjon} kan lukkes med start- og sluttdato {eldste_start} - {eldste_ny_sluttdato} "
        kanLukkesEnkelt = True 
        print( tidsvurdering )
        print( f"Nyeste versjon {minId} versjon {nyeste_versjon} med startdato                      {nyeste_start} " )
 

    # Konstruerer de dataene som skal logges: 
    sammendrag = { 'objekttype' : objekttype,
                  'nvdbId' : minId,
                  'eldsteVersjon' : eldste_versjon, 
                  'eldsteStartdato' : eldste_start,
                  'eldsteSluttdato' : eldste_slutt, 
                  'eldsteKanLukkesEnkelt' : kanLukkesEnkelt,
                  'feilmeldinger' : ','.join( list( feilmeldinger )),
                  'tidsvurdering' : tidsvurdering,
                  'eldsteNySluttdato' : eldste_ny_sluttdato,
                  'nyesteVersjon'  : nyeste_versjon,
                  'nyesteStartdato' : nyeste_start,
                  'nyesteSluttdato' : nyeste_slutt,
                 }

    # Sjekker om vi har fått gyldige data
    if len( feilmeldinger ) == 0: 
        
        # Sjekker egenskapsverdier
        analyse = objektdifferanser.sammenlignEgenskaper( [eldste, nyeste] )
        sammendrag['UlikeEgenskaper']      =  ','.join( list( analyse['avviker'].keys() ))
        sammendrag['identiskeEgenskaper']  = ','.join( list( analyse['identiske'].keys() )) 
        sammendrag['egenskapAnalyse']      = analyse

    else: 
        print( f"Feilmelding {objekttype} - {minId} : { ','.join( list( feilmeldinger )) } ")

    data.append( sammendrag  )


jajens's Password: ········
Henter 421037267 
HURRA, 421037267 versjon 1 kan lukkes med start- og sluttdato 2013-06-26 - 2013-08-08 
Nyeste versjon 421037267 versjon 2 med startdato                      2013-08-08 
Henter 421037270 
HURRA, 421037270 versjon 1 kan lukkes med start- og sluttdato 2013-06-26 - 2013-08-08 
Nyeste versjon 421037270 versjon 2 med startdato                      2013-08-08 
Henter 421037296 
HURRA, 421037296 versjon 1 kan lukkes med start- og sluttdato 2013-06-26 - 2013-08-08 
Nyeste versjon 421037296 versjon 2 med startdato                      2013-08-08 
Henter 421037269 
HURRA, 421037269 versjon 1 kan lukkes med start- og sluttdato 2013-06-26 - 2013-08-08 
Nyeste versjon 421037269 versjon 2 med startdato                      2013-08-08 
Henter 421037290 
HURRA, 421037290 versjon 1 kan lukkes med start- og sluttdato 2013-06-26 - 2013-08-08 
Nyeste versjon 421037290 versjon 2 med startdato                      2013-08-08 
Henter 421037295 
HURRA, 421037295 ve

In [8]:
# r = forb.les( '/vegobjekt', params={'id' : 325568040})

In [9]:
# obj = r.json()

In [11]:
# obj['metadata']

In [10]:
# r.json()

In [26]:
vurdering = pd.DataFrame( data)
vurdering.columns

Index(['objekttype', 'nvdbId', 'eldsteVersjon', 'eldsteStartdato',
       'eldsteSluttdato', 'eldsteKanLukkesEnkelt', 'feilmeldinger',
       'tidsvurdering', 'eldsteNySluttdato', 'nyesteVersjon',
       'nyesteStartdato', 'nyesteSluttdato', 'UlikeEgenskaper',
       'identiskeEgenskaper', 'egenskapAnalyse'],
      dtype='object')

In [27]:
vurdering.dtypes

objekttype                int64
nvdbId                    int64
eldsteVersjon             int64
eldsteStartdato          object
eldsteSluttdato          object
eldsteKanLukkesEnkelt      bool
feilmeldinger            object
tidsvurdering            object
eldsteNySluttdato        object
nyesteVersjon             int64
nyesteStartdato          object
nyesteSluttdato          object
UlikeEgenskaper          object
identiskeEgenskaper      object
egenskapAnalyse          object
dtype: object

In [14]:
vurdering['objekttype'].value_counts()

87    19
Name: objekttype, dtype: int64

In [28]:
vurdering[ vurdering['eldsteKanLukkesEnkelt'] == True ]

,objekttype,nvdbId,eldsteVersjon,eldsteStartdato,eldsteSluttdato,eldsteKanLukkesEnkelt,feilmeldinger,tidsvurdering,eldsteNySluttdato,nyesteVersjon,nyesteStartdato,nyesteSluttdato,UlikeEgenskaper,identiskeEgenskaper,egenskapAnalyse
0,87,421037267,1,2013-06-26,None,True,,"HURRA, 421037267 versjon 1 kan lukkes med star...",2013-08-08,2,2013-08-08,None,"PunktTilknytning,Assosierte Lysmast,Assosierte...","Geometri, punkt,Plassering","{'identiske': {'Geometri, punkt': {'id': 4787,..."
1,87,421037270,1,2013-06-26,None,True,,"HURRA, 421037270 versjon 1 kan lukkes med star...",2013-08-08,2,2013-08-08,None,"Assosierte Lysmast,Assosierte Lysarmatur","PunktTilknytning,Geometri, punkt,Plassering",{'identiske': {'PunktTilknytning': {'id': 1000...
2,87,421037296,1,2013-06-26,None,True,,"HURRA, 421037296 versjon 1 kan lukkes med star...",2013-08-08,2,2013-08-08,None,"Assosierte Lysmast,Assosierte Lysarmatur","PunktTilknytning,Geometri, punkt,Plassering",{'identiske': {'PunktTilknytning': {'id': 1000...
3,87,421037269,1,2013-06-26,None,True,,"HURRA, 421037269 versjon 1 kan lukkes med star...",2013-08-08,2,2013-08-08,None,"Assosierte Lysmast,Assosierte Lysarmatur","PunktTilknytning,Geometri, punkt,Plassering",{'identiske': {'PunktTilknytning': {'id': 1000...
4,87,421037290,1,2013-06-26,None,True,,"HURRA, 421037290 versjon 1 kan lukkes med star...",2013-08-08,2,2013-08-08,None,"Assosierte Lysmast,Assosierte Lysarmatur","PunktTilknytning,Geometri, punkt,Plassering",{'identiske': {'PunktTilknytning': {'id': 1000...
5,87,421037295,1,2013-06-26,None,True,,"HURRA, 421037295 versjon 1 kan lukkes med star...",2013-08-08,2,2013-08-08,None,"Assosierte Lysmast,Assosierte Lysarmatur","PunktTilknytning,Geometri, punkt,Plassering",{'identiske': {'PunktTilknytning': {'id': 1000...
6,87,421037271,1,2013-06-26,None,True,,"HURRA, 421037271 versjon 1 kan lukkes med star...",2013-08-08,2,2013-08-08,None,"Assosierte Lysmast,Assosierte Lysarmatur","PunktTilknytning,Geometri, punkt,Plassering",{'identiske': {'PunktTilknytning': {'id': 1000...
7,87,421037272,1,2013-06-26,None,True,,"HURRA, 421037272 versjon 1 kan lukkes med star...",2013-08-08,2,2013-08-08,None,"Assosierte Lysmast,Assosierte Lysarmatur","PunktTilknytning,Geometri, punkt,Plassering",{'identiske': {'PunktTilknytning': {'id': 1000...
9,87,421037266,1,2013-06-26,None,True,,"HURRA, 421037266 versjon 1 kan lukkes med star...",2013-08-08,2,2013-08-08,None,"Assosierte Lysmast,Assosierte Lysarmatur","PunktTilknytning,Geometri, punkt,Plassering",{'identiske': {'PunktTilknytning': {'id': 1000...
10,87,421037293,1,2013-06-26,None,True,,"HURRA, 421037293 versjon 1 kan lukkes med star...",2013-08-08,2,2013-08-08,None,"Assosierte Lysmast,Assosierte Lysarmatur","PunktTilknytning,Geometri, punkt,Plassering",{'identiske': {'PunktTilknytning': {'id': 1000...


In [29]:
col = list( vurdering.columns )
col = [ x for x in col if x != 'egenskapAnalyse' ]

nvdbgeotricks.skrivexcel( 'doubleversjonanalyse.xlsx', vurdering[col])

skrev 1 faner til doubleversjonanalyse.xlsx 


/mnt/c/data/leveranser/nvdbapi-V3/nvdbgeotricks.py:512: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save( )


In [17]:
len( vurdering)

19

# Detaljert rapport, avvik mellom objektversjoner

In [31]:


test = { x : []  for x in list( vurdering['objekttype'].unique()) }
debug = []

for ii, row in vurdering.iterrows(): 
    if isinstance( row['egenskapAnalyse'], dict ): 
        retval = objektdifferanser.egenskapsavvikDetaljert( row['egenskapAnalyse'], row['eldsteVersjon'] )
#         if len( retval ) > 0: 
        junk = row.to_dict()
        junk.update( retval )
        test[row['objekttype']].append( junk )
            
#     else: 
#         debug.append( row.to_dict())
                 
for objType in test: 
    test[objType] = pd.DataFrame( test[objType])
    

In [19]:
test.keys()

dict_keys([87])

In [32]:
dataFrames = []
faneNavneListe = []

for objType in list( test.keys()):
    if objType not in [524, 525] and len( test[objType] ) > 0:
        temp = test[objType].copy()
        temp.drop( columns='egenskapAnalyse', inplace=True )
        dataFrames.append( temp )
        faneNavneListe.append( str( objType ) )

nvdbgeotricks.skrivexcel( 'doubleVersionsDetaljer.xlsx', dataFrames, sheet_nameListe=faneNavneListe)

skrev 1 faner til doubleVersionsDetaljer.xlsx 


/mnt/c/data/leveranser/nvdbapi-V3/nvdbgeotricks.py:512: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save( )


# Snevrer inn "vurdering" til der eldste kan lukkes enkelt + kun stedfesting avviker

For enkel - ingen treff lenger på denne metodikken

In [15]:
# vurdering = vurdering[ vurdering['UlikeEgenskaper'].isin([ 'Liste av lokasjonsattributt', 'PunktTilknytning' ] ) ]
# vurdering = vurdering[ vurdering['eldsteKanLukkesEnkelt'] == True ]

In [16]:
# vurdering

,objekttype,nvdbId,eldsteVersjon,eldsteStartdato,eldsteSluttdato,eldsteKanLukkesEnkelt,feilmeldinger,tidsvurdering,eldsteNySluttdato,nyesteVersjon,nyesteStartdato,nyesteSluttdato,UlikeEgenskaper,identiskeEgenskaper,egenskapAnalyse


In [19]:
print( "ftid,fid,vid,nySluttdato")
for ix, row in vurdering.iterrows(): 
    print( f"{row['objekttype']},{row['nvdbId']},{row['eldsteVersjon']},{row['eldsteNySluttdato']}" )

ftid,fid,vid,nySluttdato
825,325597645,1,2012-03-27
825,325597644,1,2012-03-27
838,530892593,1,2014-07-23
487,92542003,7,2018-01-01
487,1013978118,2,2021-01-01
487,84174328,6,2018-01-01
487,290825689,4,2017-12-07
487,92541985,7,2018-01-01
87,421037268,1,2013-08-08
87,421037289,1,2013-08-08
87,421037292,1,2013-08-08
87,421037291,1,2013-08-08
87,915906260,1,2019-01-01
87,421037267,1,2013-08-08
87,421037270,1,2013-08-08
87,421037296,1,2013-08-08
87,421037269,1,2013-08-08
87,421037290,1,2013-08-08
87,421037295,1,2013-08-08
87,421037271,1,2013-08-08
87,421037272,1,2013-08-08
87,915906266,1,2019-01-01
87,421037266,1,2013-08-08
87,421037293,1,2013-08-08
87,421037264,1,2013-08-08
87,421037288,1,2013-08-08
87,421037294,1,2013-08-08
99,599083247,1,2015-06-03
99,683417531,1,2016-04-01
99,599083255,1,2015-06-03
99,599083251,1,2015-06-03
95,92363546,6,2020-10-14
95,84151863,8,2019-01-21
95,92474631,5,2016-01-01
95,92411670,6,2021-05-03
95,870329401,1,2019-09-06
95,1009168231,1,2017-04-01
95,9253310

# Gammal metodikk (funker!)

Gir også null treff 

In [20]:
temp = vurdering[ (vurdering['UlikeEgenskaper'] == '') & (vurdering['eldsteKanLukkesEnkelt'] == True)].copy()

In [21]:
len(temp)

0

In [22]:
col2 = [ 'objekttype', 'nvdbId', 'eldsteVersjon', 'eldsteStartdato',
       'eldsteNySluttdato', 'nyesteVersjon', 'nyesteStartdato'     ]

temp[col2]

,objekttype,nvdbId,eldsteVersjon,eldsteStartdato,eldsteNySluttdato,nyesteVersjon,nyesteStartdato


In [23]:
col3 = [ 'objekttype', 'nvdbId', 'eldsteVersjon', 'eldsteStartdato', 'eldsteNySluttdato', 'nyesteVersjon', 'nyesteStartdato'    ]
temp[col3]

,objekttype,nvdbId,eldsteVersjon,eldsteStartdato,eldsteNySluttdato,nyesteVersjon,nyesteStartdato


In [24]:
print( "ftid,fid,vid,nySluttdato")
for ix, row in temp.iterrows(): 
    print( f"{row['objekttype']},{row['nvdbId']},{row['eldsteVersjon']},{row['eldsteNySluttdato']}" )

ftid,fid,vid,nySluttdato


In [25]:
len( vurdering[ (vurdering['eldsteKanLukkesEnkelt'] == True)] )

23

In [120]:
myDict =  vurdering[ vurdering['nvdbId'] == 870334560]['egenskapAnalyse']

In [130]:
type( myDict.iloc[0] )

dict

In [131]:
import json
with open( 'dobbeltsjekkObjekt.json', 'w') as f: 
    json.dump(  myDict.iloc[0], f, indent=4)

# Hvilke objekter avviker kun på stedfesting? 

In [44]:
len( vurdering[ vurdering['UlikeEgenskaper'].isin([ 'Liste av lokasjonsattributt', 'PunktTilknytning' ] ) ] )

13

In [34]:
r = vurdering['UlikeEgenskaper'].value_counts()

In [43]:
r.index

Index(['PunktTilknytning,Geometri, punkt,Plassering',
       'PunktTilknytning,Assosierte Skiltplate,Assosierte Tilstand/skade, skiltpunkt,Oppsettingsutstyr',
       'Type,Bredde, enkeltlinje,Bruksområde,Plan nedfresing,Farge,Utgår_Løpemeter skravur,Vegbanereflektor,Profilering,Utgår_Areal av fylt sperreområde',
       'PunktTilknytning,Assosierte Utgår_Systemobjekt,Assosierte Skiltplate,Assosierte Tilstand/skade, skiltpunkt,Høyde over vegkant,Fundamentering,Oppsettingsutstyr,Antall oppsettingsutstyr,Miljø,Klima,Avstand fra dekkekant,Antall fundament,Eier,Vedlikeholdsansvarlig',
       'PunktTilknytning,Assosierte Skiltplate,Geometri, punkt,Fundamentering,Oppsettingsutstyr,Antall oppsettingsutstyr,Antall fundament',
       'Type,Bredde, enkeltlinje,Bruksområde,Plan nedfresing,Farge,Vegbanereflektor,Profilering',
       'PunktTilknytning,Assosierte Lysmast,Assosierte Lysarmatur,Geometri, punkt,Plassering,Bruksområde',
       'Geometri, punkt,Plassering', 'Stigning,Beregningsinfo',
     

In [58]:
objektdifferanser.objektdatoer( 421037290 )

Versjoner for NVDB objektid 421037290 type 87 Belysningspunkt  
421037290 versjon 1 2013-06-26 ----       	 https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/87/421037290/1 
421037290 versjon 2 2013-08-08 ----       	 https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/87/421037290/2 


In [95]:
data = andremangler.skrivAlleRelasjonstre( 915906266)

Versjoner for NVDB objektid 915906266 type 87 Belysningspunkt  
https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/87/915906266/1
(Ingen foreldre for objekt ID 915906266 versjon 1 )
Anders  87 Belysningspunkt 915906266 v1 2019-02-04 ----      
Anders  87 Belysningspunkt 915906266 v1 => 88 Lysarmatur 915906300 v1 2019-02-04 ----      
Anders  87 Belysningspunkt 915906266 v1 => 181 Lysmast 915906267 v1 2019-02-04 ----      
https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/87/915906266/2
(Ingen foreldre for objekt ID 915906266 versjon 2 )
Beate  87 Belysningspunkt 915906266 v2 2019-01-01 ----      
Beate  87 Belysningspunkt 915906266 v2 => 88 Lysarmatur 915906300 v1 2019-02-04 ----      
Beate  87 Belysningspunkt 915906266 v2 => 181 Lysmast 915906267 v1 2019-02-04 ----      


In [92]:
url = 'https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/99/599083249/'

In [34]:
import importlib
importlib.reload( objektdifferanser )
importlib.reload( andremangler )

<module 'andremangler' from '/mnt/c/data/leveranser/mangelrapport/andremangler.py'>

In [97]:
differ = objektdifferanser.sammenlignEgenskaper( ['https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/87/915906266/1',
                                                 'https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/87/915906266/2'])
print( json.dumps( objektdifferanser.egenskapsavvikDetaljert(differ, 1), indent=4),'\n---')

{} 
---


In [94]:
differ = objektdifferanser.sammenlignEgenskaper( ['https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/181/421763997/1',
                                                 'https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/181/421764046/1'])
print( json.dumps( objektdifferanser.egenskapsavvikDetaljert(differ, 1), indent=4),'\n---')

{} 
---
